In [ ]:
import os

import crystal_toolkit
from pymatgen.core import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.transformations.advanced_transformations import DopingTransformation

import matcalc
from matcalc import PhononCalc

## From MP

In [ ]:
api_key = os.getenv("API_KEY")

mpr = MPRester(api_key)

# fetch the structure of Zn(FeO2)2
structure_mp = mpr.get_structure_by_material_id("mp-18750")
print(structure_mp)
structure_mp

In [ ]:
sga_loose = SpacegroupAnalyzer(structure_mp, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

In [ ]:
from pymatgen.transformations.standard_transformations import SubstitutionTransformation

# Partial doping example: Replace Mn by 20%  + 80% Mn
substitution = {"Mn": {"Mn": 0.2, "Zn": 0.8}}

trans = SubstitutionTransformation(substitution)
doped_structure = trans.apply_transformation(structure_mp)

doped_structure

In [ ]:
sga_loose = SpacegroupAnalyzer(doped_structure, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

In [ ]:
from pymatgen.transformations.standard_transformations import (
    DiscretizeOccupanciesTransformation,
    OrderDisorderedStructureTransformation,
)

# Step 1: Discretize small fractional occupancies
discretizer = DiscretizeOccupanciesTransformation(tol=0.1)
s_discretized = discretizer.apply_transformation(doped_structure)

# Optional: build supercell manually to increase multiplicities
# For example, building a 2x2x2 supercell to get integer occupancies
s_supercell = s_discretized * (2, 2, 2)  # multiply unit cell axes

# Step 2: Try ordering the discretized, possibly enlarged supercell
order_trans = OrderDisorderedStructureTransformation(no_oxi_states=True)
s_ordered = order_trans.apply_transformation(s_supercell, return_ranked_list=10)

for idx, ostruct in enumerate(s_ordered):
    print(f"-----Ordering {idx + 1}-----")
    print(ostruct)
    print("\n")

doped_structure = s_ordered[0]["structure"]
doped_structure

In [ ]:
sga_loose = SpacegroupAnalyzer(doped_structure, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

In [ ]:
# Get back to R3m to fd3m

In [ ]:
calculator_2 = matcalc.load_fp("r2scan")

relax_calc = matcalc.RelaxCalc(
    calculator_2,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True
)

data = relax_calc.calc(doped_structure)

In [ ]:
final_structure_relaxed = data['final_structure']
print(final_structure_relaxed)
final_structure_relaxed

In [ ]:
sga_loose = SpacegroupAnalyzer(final_structure_relaxed, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

In [ ]:
calculator_ph = matcalc.load_fp("pbe")

phonon_calc = PhononCalc(
    calculator_ph,
    relax_structure=True,
    write_band_structure="output/MZFO_phonon_bs.yaml",
    write_total_dos="output/MZFO_phonon_dos.dat",
    write_phonon="output/MZFO_phonon.yaml",
    optimizer="FIRE",
)

data_ph = phonon_calc.calc(final_structure_relaxed)

In [ ]:
import matplotlib.pyplot as plt

phonon_bs = data_ph['phonon'].band_structure

n_axes = sum(1 for c in phonon_bs.path_connections if not c)

fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))

if n_axes == 1:
    axs = [axs]

for ax in axs[1:]:
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelleft=False)

phonon_bs.plot(ax=axs)
fig.suptitle("Phonon band structure", fontsize=16)
plt.subplots_adjust(wspace=0.07)
plt.show()


In [ ]:
phonon_dos = data_ph["phonon"].total_dos

fig, ax = plt.subplots(figsize=(8, 5))
phonon_dos.plot(ax=ax)
fig.suptitle("Phonon DOS", fontsize=16)
plt.show()

In [ ]:
plt.plot(
    data_ph['thermal_properties']['temperatures'],
    data_ph['thermal_properties']['heat_capacity']
)

plt.xlabel("Temperature (K)")
plt.ylabel('Heat Capacity (J/mol-K)')
plt.title("Heat Capacity vs Temperature")
plt.show()